In [1]:
# ✅ Step 1: Install Required Libraries
!pip install pretty_midi
!pip install numpy tensorflow pandas seaborn fluidsynth

import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import pretty_midi
import collections
import glob
import pathlib
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model

# ✅ Step 2: Download the MAESTRO MIDI Dataset
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
    tf.keras.utils.get_file(
        'maestro-v2.0.0-midi.zip',
        origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
        extract=True,
        cache_dir='.', cache_subdir='data',
    )

# ✅ Step 3: Load MIDI Files
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print(f"🎵 Number of MIDI files found: {len(filenames)}")

# ✅ Step 4: Convert MIDI to Note Sequences
def midi_to_notes(midi_file):
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start

    return pd.DataFrame(notes)

# ✅ Process First 5 MIDI Files
num_files = 5
all_notes = [midi_to_notes(f) for f in filenames[:num_files]]
all_notes = pd.concat(all_notes)
print(f"🎹 Processed {len(all_notes)} notes!")

# ✅ Step 5: Prepare Dataset for LSTM
key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)

def create_sequences(dataset, seq_length=25, vocab_size=128):
    """Create sequences for training"""
    seq_length += 1
    windows = dataset.window(seq_length, shift=1, stride=1, drop_remainder=True)
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    def split_labels(sequences):
        inputs = sequences[:-1]
        labels = {key: sequences[-1][i] for i, key in enumerate(key_order)}
        return inputs, labels

    return sequences.map(split_labels)

seq_ds = create_sequences(notes_ds)
seq_ds = seq_ds.shuffle(10000).batch(64).cache().prefetch(tf.data.AUTOTUNE)

# ✅ Step 6: Build LSTM Model
input_shape = (25, 3)
inputs = Input(shape=input_shape)
x = LSTM(128, return_sequences=True)(inputs)
x = LSTM(128)(x)

outputs = {
    'pitch': Dense(128, activation='softmax', name='pitch')(x),
    'step': Dense(1, activation='relu', name='step')(x),
    'duration': Dense(1, activation='relu', name='duration')(x),
}

model = Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss={
        'pitch': 'sparse_categorical_crossentropy',
        'step': 'mse',
        'duration': 'mse'
    },
    loss_weights={'pitch': 0.05, 'step': 1.0, 'duration': 1.0}
)

# ✅ Step 7: Train the Model
history = model.fit(seq_ds, epochs=50)

# ✅ Step 8: Generate Music 🎵
def predict_next_note(notes, model, temperature=1.0):
    inputs = tf.expand_dims(notes, 0)
    predictions = model.predict(inputs)

    pitch_logits = predictions['pitch']
    step = predictions['step']
    duration = predictions['duration']

    pitch_logits /= temperature
    pitch = tf.random.categorical(pitch_logits, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1)

    step = max(0, float(step))
    duration = max(0, float(duration))

    return int(pitch), step, duration

temperature = 1.5
num_predictions = 120
input_notes = train_notes[:25] / [128, 1, 1]
generated_notes = []

prev_start = 0
for _ in range(num_predictions):
    pitch, step, duration = predict_next_note(input_notes, model, temperature)
    start = prev_start + step
    end = start + duration
    generated_notes.append((pitch, step, duration, start, end))
    input_notes = np.delete(input_notes, 0, axis=0)
    input_notes = np.append(input_notes, [[pitch, step, duration]], axis=0)
    prev_start = start

generated_notes = pd.DataFrame(
    generated_notes, columns=['pitch', 'step', 'duration', 'start', 'end']
)

# ✅ Step 9: Convert Generated Notes to MIDI
def notes_to_midi(notes, filename="generated_music.mid"):
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    prev_start = 0
    for _, row in notes.iterrows():
        start = float(prev_start + row['step'])
        end = float(start + row['duration'])
        note = pretty_midi.Note(
            velocity=100, pitch=int(row['pitch']), start=start, end=end
        )
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    pm.write(filename)
    return pm

output_midi = notes_to_midi(generated_notes)
output_midi.write("generated_music.mid")

# ✅ Step 10: Download the MIDI File
from google.colab import files
files.download("generated_music.mid")

print("🎼 Music Generation Complete! Download 'generated_music.mid' 🎶")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=1079835cac0e70a43c58c09b4bcf4df09df362be451a00b59d4de57b47619871
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
  Preparing metadata (setup.py) ... done
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4487 sha256=7e614c2c5072193f90b8fb162848d5dc3ccb0c4ddb3af19eb14f1604f3fec7b8
  Stored in directory: /root/.cache/pip/wheels/b2/b2/ec/2f935ba979ac99f4fd20690fcb5be074859d2ab1ebeb3c0ae4
Successfully built fluidsynth
59243107/59243107 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🎵 Number of MIDI files found: 1282
🎹 Processed 18090 notes!
Epoch 1/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 33s 94ms/step - d

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


283/283 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - duration_loss: 0.2267 - loss: 0.4553 - pitch_loss: 0.1909 - step_loss: 0.0377
Epoch 3/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 26s 93ms/step - duration_loss: 0.2267 - loss: 0.4520 - pitch_loss: 0.1876 - step_loss: 0.0377
Epoch 4/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - duration_loss: 0.2267 - loss: 0.4495 - pitch_loss: 0.1851 - step_loss: 0.0377
Epoch 5/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 24s 87ms/step - duration_loss: 0.2267 - loss: 0.4497 - pitch_loss: 0.1853 - step_loss: 0.0377
Epoch 6/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 25s 86ms/step - duration_loss: 0.2267 - loss: 0.4472 - pitch_loss: 0.1828 - step_loss: 0.0377
Epoch 7/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 42s 90ms/step - duration_loss: 0.2267 - loss: 0.4460 - pitch_loss: 0.1816 - step_loss: 0.0377
Epoch 8/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - duration_loss: 0.2267 - loss: 0.4447 - pitch_loss: 0.1804 - step_loss: 0.0377
Epoch 9/50
283/283 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - duration_loss: 0.

<ipython-input-1-e2d1133ac540>:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  step = max(0, float(step))
<ipython-input-1-e2d1133ac540>:123: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  duration = max(0, float(duration))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎼 Music Generation Complete! Download 'generated_music.mid' 🎶
